In [1]:
from sbi.inference import SNPE_C

In [2]:
from sbivibm.tasks import Pyloric

In [3]:
import random
import numpy as np
import torch

seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [4]:
task = Pyloric()
prior = task.get_prior_dist()
samples, xs = task.get_precomputed_dataset()

/mnt/qb/home/macke/mgloeckler90/snlvibm/sbivibm/sbivibm/tasks/pyloric_stg/pyloric_stg.py:198: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  xs[np.isnan(xs)] = self.nan_replace


In [5]:
# In experiment3 we only use 50 000 sims in the first round thus
samples = samples[:50000]
xs = xs[:50000]

In [6]:
# We only use valid sims
mask = (xs!=-99).all(1)
samples = samples[mask]
xs = xs[mask]

In [7]:
inf = SNPE_C(prior)

In [8]:
density_estimator = inf.append_simulations(samples, xs).train(training_batch_size=1000)

Neural network successfully converged after 188 epochs.


In [9]:
posterior = inf.build_posterior()
posterior.set_default_x(task.get_observation(2))

DirectPosterior(
               method_family=snpe,
               net=<a Flow, see `.net` for details>,
               prior=BoxUniform(Uniform(low: torch.Size([31]), high: torch.Size([31])), 1),
               x_shape=torch.Size([1, 15]))
               

In [10]:
simulator = task.get_simulator(sim_type="slurm")

In [11]:
samples = posterior.sample((10000,))

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

In [12]:
xs = simulator(samples)

Completed 20/20 jobs

In [13]:
# Only valids
mask = (xs != -99).all(1)
samples = samples[mask]
xs = xs[mask]
len(xs)

2186

In [14]:
density_estimator = inf.append_simulations(samples, xs, proposal=posterior).train(training_batch_size=1000)

Using SNPE-C with atomic loss
Neural network successfully converged after 125 epochs.


In [22]:
total_num_of_needed_samples = 0
for i in range(100):
    total_num_of_needed_samples += 50000
    round2_samples = density_estimator.sample(50000, task.get_observation(2)).squeeze()
    print(prior.support.check(round2_samples).sum())
    if prior.support.check(round2_samples).sum() > 0:
        break

tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(1)


In [26]:
(1/total_num_of_needed_samples)*100

4.545454545454545e-05

In [27]:
total_num_of_needed_samples

2200000